# Predict

In [1]:
%env SM_FRAMEWORK=tf.keras

import segmentation_models as sm
#from keras_segmentation.predict import predict
import cv2
#from matplotlib import pyplot as plt
#from matplotlib import cm
#from skimage.transform import radon, rescale, rotate
#from skimage.morphology import (square, rectangle, diamond, disk, cube, octahedron, ball, octagon, star,
#                                binary_closing, closing, opening, skeletonize, erosion, dilation)
#from skimage.transform import probabilistic_hough_line
#from skimage.feature import canny
#from skimage.measure import label, regionprops, regionprops_table
from skimage.io import imread, imsave
#import math
#import pandas as pd
import numpy as np
import glob
import os
from sklearn.metrics import jaccard_score


env: SM_FRAMEWORK=tf.keras
Segmentation Models: using `tf.keras` framework.


# To use GPU

In [2]:
import tensorflow as tf

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available", len(physical_devices))

tf.config.experimental.set_memory_growth(physical_devices[0], True)


Num GPUs Available 1


# Functions

In [3]:
def make_dirs(output_dir):
    try:
        os.makedirs(output_dir)
    except OSError as error:
        print(error)
    else:
        print ("\nSuccessfully created the directory %s " % output_dir)

# detect the current working directory and print it
path = os.getcwd()
print ("The current working directory is %s" % path)


The current working directory is E:\Joao\jocival_linha_plantio_run


In [4]:
#def load_model(DECODER, BACKBONE, input_shape, encoder_weights, encoder_freeze, classes, activation):
def load_model(DECODER, model_args):
    #model = sm.Unet(
    #model = sm.Linknet(
    #model = sm.PSPNet(

    #    backbone_name=BACKBONE, input_shape=(x, x, 3), encoder_weights='imagenet',
    #    encoder_freeze=True, classes=1, activation='sigmoid'
    #)

    ## input_shape – shape of input data/image (H, W, C)
        # Unet and Linknet - H and W of input images should be divisible by factor 32.
        # PSPNet - H and W should be divisible by 6 * downsample_factor and NOT None! - downsample_factor = 8

    if DECODER == "Unet" :
        print("\n1 DECODER:", DECODER)
        model = sm.Unet(**model_args)

    elif DECODER == "Linknet":
        print("\n2 DECODER:", DECODER)
        model = sm.Linknet(**model_args)

    elif DECODER == "PSPNet" :
        print("\n3 DECODER:", DECODER)
        model = sm.PSPNet(**model_args)
    else :
        raise ValueError("Error DECODER:", DECODER)

    return model


In [5]:
def load_weights (fold_to_load = 1):
    weights_to_load = weights_path + dataset_weights + '/' + BACKBONE + '_' + \
                  DECODER + '_' + dataset_weights + '_' + str(fold_to_load) + ".hdf5"
    print("weights_to_load:", weights_to_load)

    #model = sm.Linknet(backbone_name=BACKBONE, encoder_weights='imagenet', 
    #        encoder_freeze=True, classes=1, activation='sigmoid',
    #        weights = 'dados/vgg16_linknet/vgg16_linknet_9.hdf5')

    # size - target_size?: 240 PSPNet, 256 Unet and Linknet, 288 All
    if DECODER == "Unet" or DECODER == "Linknet":
        size = 256 # 240 # 256 # 288 ## target_size
    elif DECODER == "PSPNet":
        size = 240
    else:
        raise ValueError("Error DECODER:", DECODER)

    print("\ntarget_size:", size, size)

    data_load_model_args = dict(backbone_name = BACKBONE, input_shape = (size, size, 3),
                            encoder_weights='imagenet', encoder_freeze = True,
                            classes = 1, activation = 'sigmoid', weights = weights_to_load)

    model = load_model(DECODER, data_load_model_args)
    return model
    #model = sm.Unet(
    #model = sm.Linknet(
    #model = sm.PSPNet(
    #    backbone_name = BACKBONE, encoder_weights = "imagenet", encoder_freeze = True,
    #    classes = 1, activation = "sigmoid", weights = weights_to_load)


In [6]:
#import cv2
#import numpy as np

#load images
#y_pred = cv2.imread('predictions/image_001.png')
#y_true = cv2.imread('ground_truth/image_001.png') 

# Dice similarity function
def dice(y_pred, y_true, k = 1):
    intersection = np.sum(y_pred[y_true==k]) * 2.0
    dice = intersection / (np.sum(y_pred) + np.sum(y_true))
    return dice

#dice_score = dice(y_pred, y_true, k = 255) #255 in my case, can be 1 
#print ("Dice Similarity: {}".format(dice_score))

#def dice_coef2(y_pred, y_true):
#    y_true_f = y_true.flatten()
#    y_pred_f = y_pred.flatten()
#    union = np.sum(y_true_f) + np.sum(y_pred_f)
#    if union==0: return 1
#    intersection = np.sum(y_true_f * y_pred_f)
#    return 2. * intersection / union


In [7]:
#diretorio = "D:\\Datasets\\linha_plantio\\cortadas_2152\\"
#saida = "D:\\Datasets\\linha_plantio\\cortadas_2152\\predicao/"

#diretorio = "E:\Jocival_Mapas"

#save_predict = False
save_predict = True

def make_predict():
    #nr_files = len(glob.glob(diretorio + 'images/*.tif'))
    nr_files = len(glob.glob(diretorio + dataset_predict + "/*.png"))
    print("nr_files:", nr_files)

    dados_IOU = np.zeros([nr_files, 1])
    dados_dice = np.zeros([nr_files, 1])

    result_name = BACKBONE + '_' + DECODER + "_weights_" + dataset_weights + "_predict_" + \
                  dataset_predict + '_' + str(fold_to_load)
    print("\nresult_name:", result_name)

    make_dirs(output_dir)

#     do10_Only = False
#     do10_Only = True

    i = 1
    qtd = 0
    IOU_ACC = 0.0
    dice_ACC = 0.0

    #for name in glob.glob(diretorio + 'images/*.tif'):
    for name in sorted(glob.glob(diretorio + dataset_predict + "/*.png")):
    #for name in sorted([os.path.basename(x) for x in glob.glob(os.path.join((diretorio + dataset + "/*.png")))]):
        #print("img_name:", name)
        img = imread(name) #38 484
        img = img / 255

        #print("img", img)
        #print("img.shape", img.shape) # (256, 256, 3)

        if DECODER == "PSPNet" :
            # resize image to 240, 240
            img = cv2.resize(img, (240, 240), interpolation = cv2.INTER_AREA)
            #print("img.shape", img.shape) # (240, 240, 3)

        pr = model.predict(np.array([img]))[0]
        pr = np.array(pr)
        pr = pr[:,:, 0]
        pr[pr >= 0.5] = 1
        pr[pr < 1] = 0
        pr = pr.astype('uint8')

        #mask = cv2.imread(diretorio + "labels/" + os.path.basename(name), 0)
        mask = cv2.imread(diretorio + dataset_predict + "_mask/" + os.path.basename(name), 0)

        #print("mask", mask)
        #print("mask.shape", mask.shape) # (256, 256)
        if DECODER == "PSPNet" :
            # resize image mask
            mask = cv2.resize(mask, (240, 240), interpolation = cv2.INTER_AREA)
            #print("mask.shape", mask.shape) # (240, 240)

        mask[mask > 0] = 1

        ##IOU = jaccard_score(mask.flatten(), pr.flatten(), 'binary')
        IOU = jaccard_score(mask.flatten(), pr.flatten(), average = "binary")
        #print ("IOU Similarity: {}".format(IOU))

        dice_score = dice(pr, mask, k = 1) #1 in my case, can be 255 
        #print ("Dice Similarity: {}".format(dice_score))

        #dice_score = dice_coef2(pr, mask)
        #print ("Dice Similarity: {}".format(dice_score))

        dados_IOU[qtd] = IOU
        IOU_ACC += IOU

        dados_dice[qtd] = dice_score
        dice_ACC += dice_score

        qtd += 1
        pr[pr == 1] = 255

        if save_predict :
            #print("name.split('/')[-1]:", name.split('/')[-1])

            name = name.split('/')[-1]
            name_predict = name.split('.')[0] + '_' + result_name + ".png"
            imsave(output_dir + os.path.basename(name_predict), pr, check_contrast = False)

            #print(output_dir + os.path.basename(name_predict))

            #if IOU <= 0.6: 
                #None

        print(i, end=' ')
        i += 1

#         if do10_Only and i == 11 :
#             break

    #print("\nIOU_ACC:", IOU_ACC)
    #print("IOU:", IOU)
    #print("qtd:", qtd)
    print("\nIOU_ACC / qtd:", IOU_ACC / qtd)

    #print("\ndice_ACC:", dice_ACC)
    #print("dice_score:", dice_score)
    #print("qtd:", qtd)
    print("\ndice_ACC / qtd:", dice_ACC / qtd)

    np.savetxt(output_dir + result_name + "_IOU.csv", dados_IOU, delimiter = ',')
    np.savetxt(output_dir + result_name + "_dice.csv", dados_dice, delimiter = ',')

    return dados_IOU, dados_dice


In [8]:
import csv  

def make_row (a, b, writer):
    print(a, b)
    if b == -10: # blank line
        row = [] # blank line
    elif b == -11:
        row = [a]
    else :
        row = [a, b]

    writer.writerow(row)
    #print("row:", row)
    #return row

def save_sumary(output_dir, dados_IOU, dados_dice):
    # output_dir.rsplit('/',2)[0] - remove '/fold_?/' or /fold_??/', until second /
    result_name_sumary = output_dir.rsplit('/', 2)[0] + '/' + BACKBONE + '_' + DECODER + "_weights_" + \
                         dataset_weights + "_predict_" + dataset_predict  + "_sumary.csv"

    # open the file in the write mode
    #f = open(output_dir + result_name + "_sumary.csv", 'w', encoding='UTF8')

    if fold_to_load == 1:
        mode_open = 'w'
    else:
        mode_open = 'a'

    f = open(result_name_sumary, mode_open , newline='', encoding='UTF8')

    # create the csv writer
    writer = csv.writer(f)

    #row = ["np.average(dados_IOU):", 0.00847634797536522]
    #row = ["np.average(dados_IOU):", np.average(dados_IOU)]
    #writer.writerow(row)

    fold_run = output_dir.rsplit('/', 2)[1]

    make_row('', -10, writer) # blank line

    if fold_to_load == 1:
        make_row(result_name_sumary, -11, writer)
        make_row('', -10, writer)

    make_row(fold_run, -11, writer)
    make_row('', -10, writer)

    make_row("np.average(dados_IOU):", np.average(dados_IOU), writer)
    #make_row("np.mean(dados_IOU):", np.mean(dados_IOU), writer)
    make_row("np.median(dados_IOU):", np.median(dados_IOU), writer)
    make_row("np.std(dados_IOU):", np.std(dados_IOU), writer)
    make_row("np.var(dados_IOU):", np.var(dados_IOU), writer)
    make_row('', -10, writer)

    make_row("np.average(dados_dice):", np.average(dados_dice), writer)
    #make_row("np.mean(dados_dice):", np.mean(dados_dice), writer)
    make_row("np.median(dados_dice):", np.median(dados_dice), writer)
    make_row("np.std(dados_dice):", np.std(dados_dice), writer)
    make_row("np.var(dados_dice):", np.var(dados_dice), writer)
    make_row('', -10, writer)

    # close the file
    f.close()

def load_values_from_csv():
    # Load CSV and save sumary with average/median to dice and IOU

    #import numpy as np
    ##from numpy import genfromtxt

    print("output_dir:", output_dir)

    result_name = BACKBONE + '_' + DECODER + "_weights_" + dataset_weights + "_predict_" + \
                  dataset_predict + '_' + str(fold_to_load)
    print("result_name:", result_name + "_(IOU|dice).csv\n")

    file_name = output_dir + result_name
    #file_dice_name = output_dir + result_name + "_dice.csv"

    dados_IOU = np.genfromtxt(file_name + "_IOU.csv", delimiter=',')
    dados_dice = np.genfromtxt(file_name + "_dice.csv", delimiter=',')

    return dados_IOU, dados_dice


# Run

In [ ]:
BACKBONE = "vgg16"

# 4 * 4 * 10 * 3 = 480
# 8 * 8 * 10 * 3 = 1920

#dataset_weights = "Base_A"
#dataset_weights = "Base_B"
#dataset_weights = "Base_C"
#dataset_weights = "Base_D"
#dataset_weights = "Base_E200"
#dataset_weights = "Base_E300"
#dataset_weights = "Base_E400"
dataset_weights = "Base_E500"

#dataset_predict = "Base_A" ...
#DECODER = "Unet" ...

#weights_path = "/media/sda2/home/j/Downloads/tests/results2/"
#weights_path = "/media/sda2/home/j/Downloads/tests/correct/"
weights_path = "E:/Joao/jocival_linha_plantio_run/results/"

#diretorio = "/media/sda2/home/j/Downloads/tests/Recortes/"
diretorio = "E:/Backes/Segmentacao Linha Plantio CNN/Recortes/"

folds = 10 #10

model_make_predict = True # load_weights() and make_predict()
#model_make_predict = False # load_values_from_csv()

for k in range(8): #8
    if k == 0:
        dataset_predict = "Base_A"
    elif k == 1:
        dataset_predict = "Base_B"
    elif k == 2:
        dataset_predict = "Base_C"
    elif k == 3:
        dataset_predict = "Base_D"
    elif k == 4:
        dataset_predict = "Base_E200"
    elif k == 5:
        dataset_predict = "Base_E300"
    elif k == 6:
        dataset_predict = "Base_E400"
    elif k == 7:
        dataset_predict = "Base_E500"

#for k in range(4):
#    if k == 0:
#        dataset_predict = "Base_E200"
#    elif k == 1:
#        dataset_predict = "Base_E300"
#    elif k == 2:
#        dataset_predict = "Base_E400"
#    elif k == 3:
#        dataset_predict = "Base_E500"

    for j in range(3): #3
        if j == 0:
            DECODER = "Unet"
        elif j == 1:
            DECODER = "Linknet"
        elif j == 2:
            DECODER = "PSPNet"

        for i in range(folds): #folds=10
            print("\n\tDECODER", DECODER, "dataset_weights", dataset_weights, "dataset_predict", dataset_predict)

            fold_to_load = i + 1
            print("\n# fold:", fold_to_load)
            output_dir = "results/predict/weights_" + dataset_weights + "/predict_" + dataset_predict + '/' + \
                         DECODER + "/fold_" + str(fold_to_load) + '/'

            if model_make_predict:
                model = load_weights(fold_to_load)

                dados_IOU, dados_dice = make_predict()
            else:
                dados_IOU, dados_dice = load_values_from_csv()

            save_sumary(output_dir, dados_IOU, dados_dice)

print("\n\nAll Done!\n")
#E:\Joao\jocival_linha_plantio_run\results\predict\weights_Base_A\predict_Base_A\Unet\fold_1
 #   vgg16_Unet_Base_A_1_predict_dice.csv
    
#vgg16_Unet_weights_Base_D_predict_Base_D_1_IOU
#vgg16_Unet_weights_Base_D_predict_Base_D_2_IOU
#vgg16_Unet_Base_D_predict_1_IOU

# Graph info data

In [ ]:
import pandas as pd

#diretorio = "/media/sda2/home/j/Downloads/tests/recortes_tmp/"

#DECODER = "Unet" ...

def load_csv(DECODER):
    log_to_load = weights_path + dataset + '/' + BACKBONE + '_' + DECODER + '_' + dataset + ".csv"
    print("log_to_load:", log_to_load)

    log_data = pd.read_csv(log_to_load, sep=',', engine='python')
#     print(log_data)
    return log_data


In [ ]:
import matplotlib.pyplot as plt

def make_grah(log_data, grah_info, DECODER):
    #print(log_data.keys())

    info_data = log_data[grah_info]
    val_info_data = log_data["val_" + grah_info]

    print("DECODER:", DECODER)
    print("graph:", grah_info, " - val_", grah_info)

    #loss_data = log_data["iou_score"]
    #val_loss_data = log_data["val_iou_score"]

    folds = 10
    epoch = 50

    info_mean_data = np.zeros(50)
    val_info_mean_data = np.zeros(50)

    line = 0
    for i in range (folds * epoch):
        info_mean_data[line] += info_data[i]
        val_info_mean_data[line] += val_info_data[i]

#         if line == 1:
#             print("info_data[",i,"]:", info_data[i])
#             print("val_info_data[",i,"]:", val_info_data[i])

        line += 1
        if line == 50:
            line = 0

#     print(info_mean_data)
#     print(val_info_mean_data)
#     print(info_data[50:100])

    for i in range (epoch):
        info_mean_data[i] /= 10
        val_info_mean_data[i] /= 10

    #print(info_mean_data)
    #print(val_info_mean_data)

    # summarize history for loss

#     plt.plot(log_data['loss'][0:50])
#     plt.plot(log_data['val_loss'][0:50])
#     plt.title('model loss fold 1')
#     plt.ylabel('loss')
#     plt.xlabel('epoch')
#     plt.legend(['train', 'test'], loc='upper left')
#     plt.show()

    plt.plot(info_mean_data)
    plt.plot(val_info_mean_data)
    plt.title(DECODER +' model mean ' + grah_info)

#     plt.ylabel('loss')
    plt.ylabel(grah_info)
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='best', frameon=False)

    plt.grid(axis="y")

    #plt.ylim(ymin=0.5)
    #plt.ylim(ymax=3)
    
    #plt.xlim(xmin=0)
    #plt.xlim(xmax=50)

    #output_path =  os.path.join(weights_path + dataset + '/')
    output_path = weights_path + dataset + '/grah_info/' + BACKBONE + '_' + DECODER + '_' + dataset + '_' + grah_info

    plt.savefig(output_path, dpi = 400)
    print("saved to:", output_path)

    plt.show()


# Run make_graph for the 3 networks - docoder

In [ ]:

dataset = "Base_A"
#dataset = "Base_B"
#dataset = "Base_C"
#dataset = "Base_D"
#dataset = "Base_E200"
#dataset = "Base_E300"
#dataset = "Base_E400"
#dataset = "Base_E500"

BACKBONE = "vgg16"

weights_path = "E:/Joao/jocival_linha_plantio_run/results/"

output_path = weights_path + dataset + '/grah_info/'
make_dirs(output_path)

for ii in range (3):
    if ii == 0:
        grah_info = "loss"
    elif ii == 1:
        grah_info = "iou_score"
    else :
        grah_info = "f1-score"

    for jj in range(3):
        if jj == 0:
            DECODER_graph = "Unet"
        elif jj == 1:
            DECODER_graph = "Linknet"
        else :
            DECODER_graph = "PSPNet"

        log_data_tmp = load_csv(DECODER_graph)

        make_grah(log_data_tmp, grah_info, DECODER_graph)

print("\n\nAll Done!\n")
